## Prompting Info
https://github.com/camenduru/MusicGen-colab

Some prompting info from @Duemellon

But, here's a generality:

120bpm - beats per minute. Typical dance songs are 120. Most ballads are 90-100. A real slow song is 70-85. 140+ is the kinda stuff for raves, techno, & dub

320kbps 48khz - these ensure quality of the recordings, reducing hiss & usually expanding the sound range. These numbers are just rather high for an MP3 recording but not near the values of a raw audio recording. Don't apply this to sound that is supposed to be LOFI (like Lofi hip hop) because it undoes their intent : ) 22kbps quality should start sounding like you're listening to a song through a telephone or through a megaphone

4/4, 3/4, 5/4, 2/4, etc -- This would have to get into a bit more about music theory that would be applicable here. Just know the majority of American pop songs (the huge VAST majority) is 4/4. Slow jams & waltz are 3/4 (ballads & such). 5/4 & others are more for non-US audiences that have a Spanish influence. 13/8, 11/8, 10/7, etc. "weird" numbers would be for jazz

Have fun with that stuff!

All together it looks like:

* 4/4 100bpm 320kbps 48khz motown groove
* 3/4 105bpm 320kbps 48khz piano only baroque
* 110bpm 64kbps 16khz lofi hiphop summer smooth

More info about Top-k, Top-p, Temperature and Classifier Free Guidance from ChatGPT

Top-k: Top-k is a parameter used in text generation models, including music generation models. It determines the number of most likely next tokens to consider at each step of the generation process. The model ranks all possible tokens based on their predicted probabilities, and then selects the top-k tokens from the ranked list. The model then samples from this reduced set of tokens to determine the next token in the generated sequence. A smaller value of k results in a more focused and deterministic output, while a larger value of k allows for more diversity in the generated music.

Top-p (or nucleus sampling): Top-p, also known as nucleus sampling or probabilistic sampling, is another method used for token selection during text generation. Instead of specifying a fixed number like top-k, top-p considers the cumulative probability distribution of the ranked tokens. It selects the smallest possible set of tokens whose cumulative probability exceeds a certain threshold (usually denoted as p). The model then samples from this set to choose the next token. This approach ensures that the generated output maintains a balance between diversity and coherence, as it allows for a varying number of tokens to be considered based on their probabilities.

Temperature: Temperature is a parameter that controls the randomness of the generated output. It is applied during the sampling process, where a higher temperature value results in more random and diverse outputs, while a lower temperature value leads to more deterministic and focused outputs. In the context of music generation, a higher temperature can introduce more variability and creativity into the generated music, but it may also lead to less coherent or structured compositions. On the other hand, a lower temperature can produce more repetitive and predictable music.

Classifier-Free Guidance: Classifier-Free Guidance refers to a technique used in some music generation models where a separate classifier network is trained to provide guidance or control over the generated music. This classifier is trained on labeled data to recognize specific musical characteristics or styles. During the generation process, the output of the generator model is evaluated by the classifier, and the generator is encouraged to produce music that aligns with the desired characteristics or style. This approach allows for more fine-grained control over the generated music, enabling users to specify certain attributes they want the model to capture.

These parameters, such as top-k, top-p, temperature, and classifier-free guidance, provide different ways to influence the output of a music generation model and strike a balance between creativity, diversity, coherence, and control. The specific values for these parameters can be tuned based on the desired outcome and user preferences.

## Prompts

| Good? | Prompt |
| --- | ---|
| Meh | "90s rock song with loud guitars and heavy drums" |
| Yeh| "80s pop track with bassy drums and synth" |
| Yeh | "3/4 105bpm 320kbps 48khz piano only baroque" |
| FYeh| "110bpm 320kbps 48khz synthwave retro" |

In [ ]:
# This section will run the model locally. If you are using FastAPI skip this.

from transformers import AutoProcessor, MusicgenForConditionalGeneration
import torch
import time
from sigfig import round

device = "cuda:0" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to(device)

tic = time.time()

inputs = processor(
    text=["125bpm 320kbps 48khz synthwave retro"],
    padding=True,
    return_tensors="pt",
).to(device)
audio_values = model.generate(**inputs, max_new_tokens=512).to(device)

toc = time.time()

print(f'Generation complete in {round(toc-tic, sigfigs=4)} seconds!')

In [ ]:
# If you generated a wave in the previous cell, this will allow you to play it in Jupyter

from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
# If you generated a wave above, this will save the file at the current timestamp

import scipy
from datetime import datetime

filename = f'{datetime.now().timestamp()}.wav'

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write(filename, rate=sampling_rate, data=audio_values[0, 0].cpu().numpy())

In [2]:
# This is the block you need for testing out the generation API!

import requests
from datetime import datetime

api_url = "http://127.0.0.1:8000/generate"

generate = {'prompt': '125bpm 320kbps 48khz synthwave retrowave', 'tokens': '512'}

response = requests.get(api_url, params=generate)

filename = f'{datetime.now().timestamp()}.wav'

with open(filename, 'wb') as f:
    f.write(response.content)